In [ ]:
# for google colab
!pip install traffic openap

In [21]:
import pandas as pd
from traffic.core import Traffic, Flight
import openap
import warnings

warnings.filterwarnings("ignore")

pd.options.display.max_columns = 100

In [ ]:
# file: eu_flights_2022feb20_filter_resample_meteo.parquet.gz
df_base = pd.read_parquet("https://figshare.com/ndownloader/files/55632065")
t_base = Traffic(df_base)

# file: all_optimized.parquet
df_optimized = pd.read_parquet("https://figshare.com/ndownloader/files/55634282")
t_optimized = Traffic(df_optimized)

# file: grid_era5_smoothed.parquet.gz
df_era5_cost = pd.read_parquet("https://figshare.com/ndownloader/files/55632059")

# file: grid_arpege_smoothed.parquet.gz
df_arpege_cost = pd.read_parquet("https://figshare.com/ndownloader/files/55632062")


In [ ]:
t_optimized

In [ ]:
sample_flight_ids = t_base.query(
    "timestamp.dt.hour==9 and timestamp.dt.minute==30"
).flight_ids

sample_flight_ids = t_base[sample_flight_ids].longer_than("2h").eval().flight_ids

In [ ]:
len(sample_flight_ids)

In [ ]:
# quickly get the original flight ids for filtering our selected flights
t_optimized = t_optimized.assign(
    base_flight_id=lambda d: d.flight_id.str.replace("_fuel", "")
    .str.replace("_era5_03", "")
    .str.replace("_era5_06", "")
    .str.replace("_arpege_03", "")
    .str.replace("_arpege_06", "")
)

In [ ]:
t_optimized_sampled = t_optimized.query(f"base_flight_id.isin({sample_flight_ids})")
t_optimized_sampled

## Visualizing only the flights with persistent contrails

In [ ]:
t_optimized_sampled.query("flight_id==base_flight_id").map_leaflet(zoom=4)

plot contrails segments in read over the trajectories

In [ ]:
t_optimized_sampled.query("flight_id==base_flight_id").map_leaflet(
    zoom=4,
    highlight={
        "#bd0026": lambda t: t.query("persistent"),
    },
)

## Compare with the climate sensitive regions from the ERA5 data

In [ ]:
df_era5_cost.query("timestamp.dt.hour==9")

In [ ]:
import matplotlib.pyplot as plt
from cartopy import crs as ccrs
from cartopy.feature import BORDERS


def plot_costs_grid(
    df_cost: pd.DataFrame, cmap, select_hour=10, skip_levels=5, ncols=2, nrows=2
):
    proj = ccrs.TransverseMercator(central_longitude=10, central_latitude=50)

    fig, axes = plt.subplots(
        nrows, ncols, figsize=(5 * ncols, 3.5 * nrows), subplot_kw=dict(projection=proj)
    )

    heights = df_cost.height.unique()

    for i, ax in enumerate(axes.flatten()):
        df_cost_pivot = df_cost.query(
            f"ts=={3600 * select_hour} and height=={heights[skip_levels + i]}"
        ).pivot(index="latitude", columns="longitude", values="cost")

        lat, lon, val = (
            df_cost_pivot.index.values,
            df_cost_pivot.columns.values,
            df_cost_pivot.values,
        )

        ax.set_extent([-20, 40, 28, 65])
        ax.add_feature(BORDERS, lw=0.5, edgecolor="gray")
        ax.coastlines(resolution="110m", lw=0.5, color="gray")
        ax.gridlines(
            draw_labels=False, linewidth=0.5, color="gray", alpha=0.5, linestyle="--"
        )
        ax.contourf(
            lon,
            lat,
            val,
            cmap=cmap,
            alpha=0.8,
            transform=ccrs.PlateCarree(),
            vmin=-df_cost.cost.quantile(0.01),
            vmax=df_cost.cost.quantile(0.99),
        )

        fl = heights[skip_levels + i] / openap.aero.ft // 1000 * 10

        ax.text(0.03, 0.9, f"FL{int(fl)}", transform=ax.transAxes, fontsize=20)

    plt.tight_layout()
    return plt


In [ ]:
plot_costs_grid(df_era5_cost, select_hour=9, cmap="Reds")

## How does the forecast data look like when dealing with persistent contrails?

In [ ]:
plot_costs_grid(df_arpege_cost, select_hour=9, cmap="RdPu")